# Exploring and Cleaning the data

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_colwidth', None)

In [2]:
data = pd.read_csv('train_aug_in_kag_org.csv')

In [3]:
data.head(10)

,title,text,title_text,label
0,squirrel financial wellbeing,A Squirrel account makes it easy to stick to your budget and save. Start for free today!,squirrel financial wellbeing. A Squirrel account makes it easy to stick to your budget and save. Start for free today!,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
1,senior insurance marketing,"Welcome to the home page of Senior Insurance Marketing, a full service brokerage specializing in Medicare and senior insurance products. Located in Lincoln, NE.","senior insurance marketing. Welcome to the home page of Senior Insurance Marketing, a full service brokerage specializing in Medicare and senior insurance products. Located in Lincoln, NE.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
2,3i financial group inc.,"A Small Boutique Firm Dedicated to Achieving Our Clients Goals.#sep#inform.invest.insure Get a Free, No Obligation Quote forAllYour Insurance Needs. 3i Financial offers a wide variety of insurance products to help suit your needs. Choose from individual, family, and group. GET A QUOTE TODAY Travel Insurance Get peace of mind with comprehensive travel insurance. Get a free quote today! Health & Dental Get affordable health Read more","3i financial group inc.. A Small Boutique Firm Dedicated to Achieving Our Clients Goals.#sep#inform.invest.insure Get a Free, No Obligation Quote forAllYour Insurance Needs. 3i Financial offers a wide variety of insurance products to help suit your needs. Choose from individual, family, and group. GET A QUOTE TODAY Travel Insurance Get peace of mind with comprehensive travel insurance. Get a free quote today! Health & Dental Get affordable health Read more",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
3,brownstone insurance,"Brownstone Insurance provides master insurance policies for multi-unit buildings throughout Massachusetts, as well as personal insurance products such as renter's insurance and homeowner's insurance.#sep#Brownstone Insurance provides master insurance policies for multi-unit \nbuildings throughout Massachusetts, as well as personal insurance products \nsuch as renter's insurance and homeowner's insurance.","brownstone insurance. Brownstone Insurance provides master insurance policies for multi-unit buildings throughout Massachusetts, as well as personal insurance products such as renter's insurance and homeowner's insurance.#sep#Brownstone Insurance provides master insurance policies for multi-unit \nbuildings throughout Massachusetts, as well as personal insurance products \nsuch as renter's insurance and homeowner's insurance.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
4,pacificwide lending,Learn more about Pacificwide Lending and our commercial & residential loan services in California.,pacificwide lending. Learn more about Pacificwide Lending and our commercial & residential loan services in California.,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
5,city auto finance,"City Auto Finance is the Southeast's most trusted used-car floor plan financing company with locations in Memphis, TN; Mobile, AL; Huntsville, AL and Murfreesboro, TN. We provide you with competitive rates and the capability to make payments online.","city auto finance. City Auto Finance is the Southeast's most trusted used-car floor plan financing company with locations in Memphis, TN; Mobile, AL; Huntsville, AL and Murfreesboro, TN. We provide you with competitive rates and the capability to make payments online.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
6,"veterans financial, inc.","VA Aid and Attendance benefit can help pay for assisted living, home care or nursing home. Veterans Financial Inc can advise you at no cost about benefits","veterans financial, inc.. VA Aid and Attendance benefit can help pay for assisted living, home care or nursing home. Veterans Financial Inc can advise you at no cost 

In [4]:
data.shape

(34953, 4)

In [5]:
data.isnull().sum()

title         42
text           0
title_text     0
label          0
dtype: int64

In [6]:
data.dropna(inplace=True)

In [7]:
data['label'].nunique()

15

In [8]:
data['label'].value_counts()

payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing                                                 6160
api-code-technical-gear-parameters-setup-config                                                                       6145
hospital-health-safety-doctor-nurse-medicine-medical-clinic                                                           5634
edit-paint-art-palette-lightbulb-idea-image                                                                           5043
restaurant-food-eat-eating people-water-drink                                                                         2161
building-appartment-house-block-construction-kitchen-pool-parking-automation-decoration-decor-furniture-room-table    1957
education                                                                                                             1853
beauty-fashion-makeup-women                                                                                           1802
travel-hospitali

In [9]:
all(data['text']==data['title_text'])

False

In [10]:
sum(data['text']==data['title_text'])

27

In [11]:
data['len_text'] = data['text'].apply(lambda x: len(x))
data['len_title_text'] = data['title_text'].apply(lambda x: len(x))

In [12]:
all(data['len_title_text']>data['len_text'])

False

In [13]:
sum(data['len_title_text']>data['len_text'])

33297

In [14]:
data.drop(columns=['text', 'len_text', 'len_title_text'], inplace=True)

In [15]:
data.head()

,title,title_text,label
0,squirrel financial wellbeing,squirrel financial wellbeing. A Squirrel account makes it easy to stick to your budget and save. Start for free today!,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
1,senior insurance marketing,"senior insurance marketing. Welcome to the home page of Senior Insurance Marketing, a full service brokerage specializing in Medicare and senior insurance products. Located in Lincoln, NE.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
2,3i financial group inc.,"3i financial group inc.. A Small Boutique Firm Dedicated to Achieving Our Clients Goals.#sep#inform.invest.insure Get a Free, No Obligation Quote forAllYour Insurance Needs. 3i Financial offers a wide variety of insurance products to help suit your needs. Choose from individual, family, and group. GET A QUOTE TODAY Travel Insurance Get peace of mind with comprehensive travel insurance. Get a free quote today! Health & Dental Get affordable health Read more",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
3,brownstone insurance,"brownstone insurance. Brownstone Insurance provides master insurance policies for multi-unit buildings throughout Massachusetts, as well as personal insurance products such as renter's insurance and homeowner's insurance.#sep#Brownstone Insurance provides master insurance policies for multi-unit \nbuildings throughout Massachusetts, as well as personal insurance products \nsuch as renter's insurance and homeowner's insurance.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
4,pacificwide lending,pacificwide lending. Learn more about Pacificwide Lending and our commercial & residential loan services in California.,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing


# Preparing our data for modeling 

In [16]:
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
# title
title_corpus = []
lm = WordNetLemmatizer()
words = stopwords.words('english')

for i in range(data.shape[0]):
    row = re.sub('[^a-zA-Z\s]', '', data.iloc[i, 0]).lower()
    row = row.split()  
    row = [lm.lemmatize(word) for word in row if word not in words]
    row = ' '.join(row)
    title_corpus.append(row)

In [18]:
len(title_corpus)

34911

In [19]:
title_corpus[0]

'squirrel financial wellbeing'

In [20]:
# text
text_corpus = []
lm = WordNetLemmatizer()
words = stopwords.words('english')

for i in range(data.shape[0]):
    row = re.sub('[^a-zA-Z\s]', '', data.iloc[i, 1]).lower()
    row = row.split()  
    row = [lm.lemmatize(word) for word in row if word not in words]
    row = ' '.join(row)
    text_corpus.append(row)

In [21]:
len(text_corpus)

34911

In [22]:
text_corpus[0]

'squirrel financial wellbeing squirrel account make easy stick budget save start free today'

_Our Bag of Words_

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

In [24]:
title_cv = CountVectorizer(max_features=200)   
X_title = title_cv.fit_transform(title_corpus).toarray()

In [25]:
X_title.shape

(34911, 200)

In [26]:
text_cv = CountVectorizer(max_features=800)   
X_text = text_cv.fit_transform(text_corpus).toarray()

In [27]:
X_text.shape

(34911, 800)

In [28]:
X = np.concatenate((X_title, X_text), axis=1)
y = data.iloc[:, 2].values

In [29]:
X.shape, y.shape

((34911, 1000), (34911,))

In [30]:
# LabelEncoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

In [31]:
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

# Splitting the data

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [33]:
X_train.shape, X_val.shape

((27928, 1000), (6983, 1000))

# Modeling

In [34]:
from time import time 

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier 

from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

In [ ]:
models = [
    ('GradientBoostingClassifier',GradientBoostingClassifier()),
    ('XGBoost', XGBClassifier(use_label_encoder=False, eval_metric='logloss')),
    ('LightGBM', LGBMClassifier())
]

results = pd.DataFrame(columns = ['Model', 'Accuracy', 'Precision-MacroAVG', 'Precision-WeightedAVG', 
                                  'Recall-MacroAVG', 'Recall-WeightedAVG', 'F1_score-MacroAVG','F1_score-WeightedAVG', 'Time_taken'])
for model_name, model in models:
    start = time()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    end = time()
    
    record = {}
    record['Model'] = model_name
    record['Accuracy'] = accuracy_score(y_val, y_pred)*100
    record['Precision-MacroAVG'] = precision_score(y_val, y_pred, average='macro')*100
    record['Precision-WeightedAVG'] = precision_score(y_val, y_pred, average='weighted')*100
    record['Recall-MacroAVG'] = recall_score(y_val, y_pred, average='macro')*100
    record['Recall-WeightedAVG'] = recall_score(y_val, y_pred, average='weighted')*100
    record['F1_score-MacroAVG'] = f1_score(y_val, y_pred, average='macro')*100
    record['F1_score-WeightedAVG'] = f1_score(y_val, y_pred, average='weighted')*100
    record['Time_taken'] = end-start
    
    results = results.append(record, ignore_index=True)
    
results = results.sort_values(by='Accuracy', ascending=False)

In [39]:
results

,Model,Accuracy,Precision-MacroAVG,Precision-WeightedAVG,Recall-MacroAVG,Recall-WeightedAVG,F1_score-MacroAVG,F1_score-WeightedAVG,Time_taken
2,LightGBM,89.918373,86.454751,89.951421,81.808739,89.918373,83.671041,89.838809,61.854151
0,GradientBoostingClassifier,86.696262,83.544238,86.623048,76.089862,86.696262,78.763896,86.400653,1752.563301
1,XGBoost,84.591150,83.296182,84.508368,68.903998,84.591150,72.558795,83.961185,800.352957


In [40]:
# saving the models
import joblib

joblib.dump(models[0][1], 'GradientBoostingClassifier.sav')
joblib.dump(models[1][1], 'XGBoost.sav')
joblib.dump(models[2][1], 'LightGBM.sav')

# Testing on test_data

In [35]:
test_data = pd.read_csv('test_in_kag_org.csv')

In [36]:
test_data.shape

(6501, 4)

In [37]:
test_data.isnull().sum()

title         0
text          0
title_text    0
label         0
dtype: int64

In [38]:
test_data['label'].nunique()

15

In [39]:
test_data.drop(columns=['text'], inplace=True)

# Preparing our data for modeling 

In [40]:
# title
title_corpus_test = []
lm = WordNetLemmatizer()
words = stopwords.words('english')

for i in range(test_data.shape[0]):
    row = re.sub('[^a-zA-Z\s]', '', test_data.iloc[i, 0]).lower()
    row = row.split()  
    row = [lm.lemmatize(word) for word in row if word not in words]
    row = ' '.join(row)
    title_corpus_test.append(row)

In [41]:
len(title_corpus_test)

6501

In [42]:
# text
text_corpus_test = []
lm = WordNetLemmatizer()
words = stopwords.words('english')

for i in range(test_data.shape[0]):
    row = re.sub('[^a-zA-Z\s]', '', test_data.iloc[i, 1]).lower()
    row = row.split()  
    row = [lm.lemmatize(word) for word in row if word not in words]
    row = ' '.join(row)
    text_corpus_test.append(row)

In [43]:
len(text_corpus_test)

6501

In [44]:
text_corpus_test[0]

'centralized hub procore offer ability communicate team get full view project one place assign task stay connected google map google doc'

_Our Bag of Words_

In [45]:
X_title_test = title_cv.transform(title_corpus_test).toarray()

In [46]:
X_title_test.shape

(6501, 200)

In [47]:
X_text_test = text_cv.transform(text_corpus_test).toarray()

In [48]:
X_text_test.shape

(6501, 800)

In [49]:
X_test = np.concatenate((X_title_test, X_text_test), axis=1)
y_test = test_data.iloc[:, 2].values

In [50]:
X_test.shape, y_test.shape

((6501, 1000), (6501,))

In [51]:
y_test = le.transform(y_test)

In [52]:
np.unique(y_test)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [53]:
import joblib

GB_model = joblib.load('GradientBoostingClassifier.sav')
XGB_model = joblib.load('XGBoost.sav') 
LGBM_model = joblib.load('LightGBM.sav')

In [54]:
models = [
    ('GradientBoostingClassifier',GB_model),
    ('XGBoost', XGB_model),
    ('LightGBM', LGBM_model)
]

results = pd.DataFrame(columns = ['Model', 'Accuracy', 'Precision-MacroAVG', 'Precision-WeightedAVG', 
                                  'Recall-MacroAVG', 'Recall-WeightedAVG', 'F1_score-MacroAVG','F1_score-WeightedAVG'])
for model_name, model in models:
    y_pred = model.predict(X_test)
    
    record = {}
    record['Model'] = model_name
    record['Accuracy'] = accuracy_score(y_test, y_pred)*100
    record['Precision-MacroAVG'] = precision_score(y_test, y_pred, average='macro')*100
    record['Precision-WeightedAVG'] = precision_score(y_test, y_pred, average='weighted')*100
    record['Recall-MacroAVG'] = recall_score(y_test, y_pred, average='macro')*100
    record['Recall-WeightedAVG'] = recall_score(y_test, y_pred, average='weighted')*100
    record['F1_score-MacroAVG'] = f1_score(y_test, y_pred, average='macro')*100
    record['F1_score-WeightedAVG'] = f1_score(y_test, y_pred, average='weighted')*100
    
    results = results.append(record, ignore_index=True)
    
results = results.sort_values(by='Accuracy', ascending=False)
results

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Pr

,Model,Accuracy,Precision-MacroAVG,Precision-WeightedAVG,Recall-MacroAVG,Recall-WeightedAVG,F1_score-MacroAVG,F1_score-WeightedAVG
2,LightGBM,90.662975,72.680326,90.486304,67.908639,90.662975,68.606829,90.423365
0,GradientBoostingClassifier,89.493924,68.302227,89.417484,72.219715,89.493924,68.839056,89.308907
1,XGBoost,89.109368,68.010093,89.020311,68.639421,89.109368,66.884663,88.833283
